In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "f5b7c5ed-8ebd-48d3-99fc-c77283bf5cd2",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "source": [
    "## Load Data to delta table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "d571b531-89ac-42df-bfd9-a6dc6f74c155",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Copy the file from the local filesystem to DBFS\n",
    "dbutils.fs.cp(\n",
    "    'file:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv',\n",
    "    'dbfs:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv'\n",
    ")\n",
    "\n",
    "# Load data to DataFrame\n",
    "data = spark.read.format(\"csv\").option(\"header\", \"true\").load(\n",
    "    \"dbfs:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv\"\n",
    ")\n",
    "\n",
    "# Rename columns to remove invalid characters\n",
    "for col in data.columns:\n",
    "    new_col = col.replace(' ', '_').replace(';', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('\\n', '').replace('\\t', '').replace('=', '')\n",
    "    data = data.withColumnRenamed(col, new_col)\n",
    "\n",
    "# Write data to Delta table\n",
    "data.write.format(\"delta\").mode(\"overwrite\").save(\"dbfs:/delta/amazon_sale_report\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "a2b38383-813c-4aed-bac3-18e8cd60bd6c",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Read data from Delta table and display it\n",
    "delta_table = spark.read.format(\"delta\").load(\"dbfs:/delta/amazon_sale_report\")\n",
    "delta_table.display()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "1362f0b3-3e86-4f3d-8702-b487b104850e",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "source": [
    "Clean Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "533a6cc3-e99b-46a4-ae31-b544bd12c55d",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Remove column Unnamed:_22\n",
    "data = data.drop(\"Unnamed:_22\")\n",
    "\n",
    "# Overwrite the Delta table with the updated DataFrame\n",
    "data.write.format(\"delta\").mode(\"overwrite\").save(\"dbfs:/delta/amazon_sale_report\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "28449517-9694-4bb9-b793-2ee313860644",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Read data from Delta table and display it\n",
    "display(delta_table)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "c4580ef0-5dc0-4681-8218-243cc176a8d1",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%fs ls 'dbfs:/delta/amazon_sale_report/_delta_log'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "1bb9af73-53f1-4f98-90ae-5c9e3f45e551",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%fs head 'dbfs:/delta/amazon_sale_report/_delta_log/00000000000000000001.json'"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "be5a46e6-f353-4cc7-84f7-d79ff4f0f147",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "source": [
    "# SQL"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "e35a9ca3-d504-4a05-bc6b-99d47bf958fe",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "# Create a table in the demo_databricks database using the delta_table DataFrame\n",
    "delta_table.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"demo_databricks.default.delta_table\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "3aa81de6-f172-4e54-80f5-a0dc95c15a3e",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "SELECT * FROM demo_databricks.default.delta_table LIMIT 10"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "ee4bf75a-d005-4350-8d05-6dc0d8e85358",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "DESCRIBE HISTORY delta_table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "83ffb9f6-d6bc-4eb7-839b-50d85f5126f1",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "UPDATE delta_table\n",
    "SET Amount = 0.0\n",
    "WHERE Amount IS NULL;\n",
    "\n",
    "SELECT * FROM delta_table LIMIT 10"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "4a4efb84-9914-43af-9fb5-e44a3e85545c",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "CREATE VIEW DressCategoryView AS\n",
    "SELECT *\n",
    "FROM delta_table\n",
    "WHERE Category LIKE '%Dress%'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "91d45c36-594c-44c3-b30e-392e2944e6ef",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "SELECT * FROM DressCategoryView LIMIT 10"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "aa06ea1e-e0ca-4925-b7cb-ff9c9b6b7c59",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "SHOW TABLES"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {},
     "inputWidgets": {},
     "nuid": "955ae12e-78de-42e3-856b-81e7cb2600fe",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "DROP TABLE"
   ]
  }
 ],
 "metadata": {
  "application/vnd.databricks.v1+notebook": {
   "computePreferences": null,
   "dashboards": [],
   "environmentMetadata": {
    "base_environment": "",
    "environment_version": "2"
   },
   "inputWidgetPreferences": null,
   "language": "python",
   "notebookMetadata": {
    "mostRecentlyExecutedCommandWithImplicitDF": {
     "commandId": 8762923621401916,
     "dataframes": [
      "_sqldf"
     ]
    },
    "pythonIndentUnit": 4
   },
   "notebookName": "Delta Table Data Loading and Manipulation",
   "widgets": {}
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}